In [1]:
# Imports

import pandas as pd
import numpy as np
from tensorflow import keras
import sklearn # for model evaluation

from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layers.

from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics

In [2]:
# Gets the data from local drive

file_name = "chess_games.csv"
chess_data = pd.read_csv(file_name)
chess_data

,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
5,Blitz tournament,lucaseixasouza,diguim,0-1,2016.06.30,22:00:02,1773,1809,-10.0,10.0,C27,Vienna Game,180+0,Normal,1. e4 e5 2. Nc3 d6 3. Nf3 h6 4. Bc4 c6 5. b3 Q...
6,Blitz tournament,RENZZO77,HeadlessChicken,0-1,2016.06.30,22:00:02,1895,1886,-12.0,12.0,B10,Caro-Kann Defense: Two Knights Attack,180+0,Time forfeit,1. e4 c6 2. Nf3 d5 3. Nc3 dxe4 4. Nxe4 Nf6 5. ...
7,Blitz tournament,ipero,Bayern123,1-0,2016.06.30,22:00:02,2155,2356,20.0,-20.0,D02,Queen's Pawn Game: London System,180+0,Normal,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
8,Blitz tournament,Loginov19510410,Kereshu,0-1,2016.06.30,22:00:02,2010,2111,-9.0,9.0,A45,Indian Game,300+0,Normal,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
9,Blitz tournament,Shambobala,cernunnoss,1-0,2016.06.30,22:00:02,1764,1773,12.0,-12.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Nf6 4. Be2 c6 5. ...


In [3]:
# Cleans up the data by taking only the columns we want

chess_clean = chess_data[['Event', 'Result','Termination','AN']]

# Also cleans up rows to not include stalemates
chess_clean = chess_clean[chess_clean['Result']!="1/2-1/2"]

# And non-classical games
chess_clean = chess_clean[chess_clean['Event'] == " Classical "]

# And finally only normal termination. No time-out wins
chess_clean = chess_clean[chess_clean['Termination'] == "Normal"]
chess_clean = chess_clean.reset_index()


# Gets rid of weirdly annotated "AN" rows
chess_clean = chess_clean[~chess_clean['AN'].str.contains("eval")]

# And only takes checkmates
chess_clean = chess_clean[chess_clean['AN'].str.contains("#")]

chess_clean

# Figure out what the key was for "Classical"
#chess_clean['Event'].unique()

# This prunes the dataset from 6 million records to 1 million

,index,Event,Result,Termination,AN
2,59,Classical,1-0,Normal,1. e4 e5 2. Bc4 Nc6 3. Nf3 Bc5 4. c3 Nf6 5. d4...
13,129,Classical,1-0,Normal,1. d4 e6 2. e4 Bb4+ 3. Nc3 Bxc3+ 4. bxc3 Nf6 5...
16,137,Classical,0-1,Normal,1. d4 e6 2. f3 Nc6 3. c3 d5 4. e4 dxe4 5. fxe4...
18,147,Classical,0-1,Normal,1. b4 e5 2. Ba3 Nc6 3. b5 Na5 4. Nc3 a6 5. Ne4...
19,161,Classical,0-1,Normal,1. e4 e5 2. Bc4 h6 3. Nf3 Nf6 4. Nxe5 Qe7 5. B...
20,162,Classical,0-1,Normal,1. e4 c5 2. a3 d5 3. d3 b5 4. Be2 Bb7 5. Bh5 g...
24,175,Classical,1-0,Normal,1. e4 e6 2. Bc4 Qg5 3. Nf3 Qxg2 4. Rg1 Qh3 5. ...
26,190,Classical,0-1,Normal,1. e4 e5 2. f4 exf4 3. Nf3 g6 4. d4 Bg7 5. Bxf...
28,216,Classical,1-0,Normal,1. c3 d5 2. e3 Nf6 3. Nf3 Nc6 4. Bb5 Bg4 5. Bx...
35,257,Classical,1-0,Normal,1. e4 Nc6 2. Bc4 e6 3. c3 Nf6 4. d3 Ne5 5. Bb3...


# Creates helper methods for creating a playable game state given specific moves #

In [4]:
# First set up understandings of what the gamestate is
"""
Board is 8x8

0,0  1,1  0,2  0,3  0,4  0,5  0,6  0,7          A1,  B1,  C1,  D1,  E1,  F1,  G1,  H1
1,0  1,1  1,2  1,3  1,4  1,5  1,6  1,7          A2,  B2,  C2,  D2,  E2,  F2,  G2,  H2
2,0  2,1  2,2  2,3  2,4  2,5  2,6  2,7          A3,  B3,  C3,  D3,  E3,  F3,  G3,  H3
3,0  3,1  3,2  3,3  3,4  3,5  3,6  3,7          A4,  B4,  C4,  D4,  E4,  F4,  G4,  H4
4,0  4,1  4,2  4,3  4,4  4,5  4,6  4,7   ===    A5,  B5,  C5,  D5,  E5,  F5,  G5,  H5
5,0  5,1  5,2  5,3  5,4  5,5  5,6  5,7          A6,  B6,  C6,  D6,  E6,  F6,  G6,  H6
6,0  6,1  6,2  6,3  6,4  6,5  6,6  6,7          A7,  B7,  C7,  D7,  E7,  F7,  G7,  H7
7,0  7,1  7,2  7,3  7,4  7,5  7,6  7,7          A8,  B8,  C8,  D8,  E8,  F8,  G8,  H8

Pieces are

Empty Space = 0

White Pieces                   Black Pieces
Pawn = 1                       Pawn = 7
Knight = 2                     Knight = 8
Bishop = 3                     Bishop = 9
Rook = 4                       Rook = 10
Queen = 5                      Queen = 11
King = 6                       King = 12


So a starting board should look like:

4  2  3  5  6  3  2  4
1  1  1  1  1  1  1  1
0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0
7  7  7  7  7  7  7  7
10 8  9  11 12 9  8  10
"""

'\nBoard is 8x8\n\n0,0  1,1  0,2  0,3  0,4  0,5  0,6  0,7          A1,  B1,  C1,  D1,  E1,  F1,  G1,  H1\n1,0  1,1  1,2  1,3  1,4  1,5  1,6  1,7          A2,  B2,  C2,  D2,  E2,  F2,  G2,  H2\n2,0  2,1  2,2  2,3  2,4  2,5  2,6  2,7          A3,  B3,  C3,  D3,  E3,  F3,  G3,  H3\n3,0  3,1  3,2  3,3  3,4  3,5  3,6  3,7          A4,  B4,  C4,  D4,  E4,  F4,  G4,  H4\n4,0  4,1  4,2  4,3  4,4  4,5  4,6  4,7   ===    A5,  B5,  C5,  D5,  E5,  F5,  G5,  H5\n5,0  5,1  5,2  5,3  5,4  5,5  5,6  5,7          A6,  B6,  C6,  D6,  E6,  F6,  G6,  H6\n6,0  6,1  6,2  6,3  6,4  6,5  6,6  6,7          A7,  B7,  C7,  D7,  E7,  F7,  G7,  H7\n7,0  7,1  7,2  7,3  7,4  7,5  7,6  7,7          A8,  B8,  C8,  D8,  E8,  F8,  G8,  H8\n\nPieces are\n\nEmpty Space = 0\n\nWhite Pieces                   Black Pieces\nPawn = 1                       Pawn = 7\nKnight = 2                     Knight = 8\nBishop = 3                     Bishop = 9\nRook = 4                       Rook = 10\nQueen = 5                      Queen

In [5]:
# Helper method to create a brand new game_state

def setUpGame():
    game_state_set = np.zeros(shape=(8,8))
    
    # Sets up pawns
    game_state_set[1][:] = 1
    game_state_set[6][:] = 7
    
    # Sets up Rooks
    game_state_set[0][0] = 4
    game_state_set[0][7] = 4
    game_state_set[7][0] = 10
    game_state_set[7][7] = 10
    
    # Sets up Knights
    game_state_set[0][1] = 2
    game_state_set[0][6] = 2
    game_state_set[7][1] = 8
    game_state_set[7][6] = 8
    
    # Sets up Bishops
    game_state_set[0][2] = 3
    game_state_set[0][5] = 3
    game_state_set[7][2] = 9
    game_state_set[7][5] = 9
    
    # Sets up Queens
    game_state_set[0][3] = 5
    game_state_set[7][3] = 11
    
    # Sets up Kings
    game_state_set[0][4] = 6
    game_state_set[7][4] = 12
    
    return game_state_set

In [6]:
# First we create a helper method for pawn movement

# isWhite is a boolean value to see which color is moving
# move is the string of the move in Algebraic Notation
# game_state is the board
def move_pawn(isWhite, move, game_state):
    if(isWhite):
        pawn = 1
    else:
        pawn = 7
        
    # Get the file of the pawn moving
    file = ord(move[0])-97
    
    #  Checks how many pawns are on this file
    countPawns = 0
    
    for i in range(7):
        if(game_state[i][file] == pawn):
            countPawns += 1
    
    # If only 1 pawn, easy
    if(countPawns == 1):
        
        # Check if pawn is capturing or not first
        if(move[1] == ('x')):
            new_file = ord(move[2])-97
            new_rank = ord(move[3])-49
            
            # Check for an en passant first
            if(game_state[new_rank][new_file] == 0):
                if(isWhite):
                    game_state[new_rank-1][new_file] = 0
                else:
                    game_state[new_rank+1][new_file] = 0
        else:
            new_file = file
            new_rank = ord(move[1])-49
            
        # Find position of pawn
        for i in range(7):
            if(game_state[i][file] == pawn):
                rank = i
                break
        
        # Codify where pawn moved from
        game_state[rank][file] *= -1
        
        # Codify where pawn moved to
            
        game_state[new_rank][new_file] = pawn
        
        # Strips move to the possibility of promotion
        move = move[1:]
        
        if(move[0] == "x"):
            move = move[2:]
            
        move = move[1:]
        
        if(len(move) != 0):
            # Checks if pawn promoted in the move too
            if(move[0] == "="):
                if(move[1] == "Q"):
                    game_state[new_rank][new_file] = 5
                elif(move[1] == "R"):
                    game_state[new_rank][new_file] = 4
                elif(move[1] == "B"):
                    game_state[new_rank][new_file] = 3
                else:
                    game_state[new_rank][new_file] = 2

                # Also checks which color is moving to get the correct color piece
                if(not isWhite):
                    game_state[new_rank][new_file] += 6
    else:
        # Checks if pawn is only moving
        if(not("x" in move)):
            file = ord(move[0])-97
            new_file = file
            new_rank = ord(move[1]) - 49
            
            # Since pawn is only moving, pawn should be very close to the new rank
            # Checks which color is moving to know which direction to backtrack
            if(isWhite):
                # Normal move
                if(game_state[new_rank-1][new_file] == pawn):
                    game_state[new_rank-1][new_file] *= -1
                    game_state[new_rank][new_file] = pawn
                # En Passant
                else:
                    game_state[new_rank-2][new_file] *= -1
                    game_state[new_rank][new_file] = pawn
            else:
                # Normal move
                if(game_state[new_rank+1][new_file] == pawn):
                    game_state[new_rank + 1][new_file] *= -1
                    game_state[new_rank][new_file] = pawn
                # En Passant
                else:
                    game_state[new_rank+2][new_file] *= -1
                    game_state[new_rank][new_file] = pawn
            
            # Strips move to the possibility of promotion
            move = move[1:]

            if(move[0] == "x"):
                move = move[2:]

            move = move[1:]

            if(len(move) != 0):
                # Checks if pawn promoted in the move too
                if(move[0] == "="):
                    if(move[1] == "Q"):
                        game_state[new_rank][new_file] = 5
                    elif(move[1] == "R"):
                        game_state[new_rank][new_file] = 4
                    elif(move[1] == "B"):
                        game_state[new_rank][new_file] = 3
                    else:
                        game_state[new_rank][new_file] = 2

                    # Also checks which color is moving to get the correct color piece
                    if(not isWhite):
                        game_state[new_rank][new_file] += 6
                        
        # Otherwise pawn is capturing, which makes this more difficult
        else:
            file = ord(move[0])-97
            new_file = ord(move[2])-97
            new_rank = ord(move[3])-49
            
            # Check which direction a piece would be taking by looking at the color it is
            if(isWhite):
                rank = new_rank - 1
            else:
                rank = new_rank + 1
            
            # Check for an en passant first
            if(game_state[new_rank][new_file] == 0):
                if(isWhite):
                    game_state[new_rank-1][new_file] = 0
                else:
                    game_state[new_rank+1][new_file] = 0
            
            game_state[rank][file] *= -1
            game_state[new_rank][new_file] = pawn
            
            # Checks if pawn promoted in the move too
            if(move[-2] == "="):
                if(move[-1] == "Q"):
                    game_state[new_rank][new_file] = 5
                elif(move[-1] == "R"):
                    game_state[new_rank][new_file] = 4
                elif(move[-1] == "B"):
                    game_state[new_rank][new_file] = 3
                else:
                    game_state[new_rank][new_file] = 2

                # Also checks which color is moving to get the correct color piece
                if(not isWhite):
                    game_state[new_rank][new_file] += 6
    return game_state

In [7]:
# Create a helper method to find the King in question

def find_king(isWhite, move, game_state):
    if(isWhite):
        king = 6
    else:
        king = 12
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
    
    new_file = ord(move[0])-97
    new_rank = ord(move[1])-49
    
    # Checks to make sure it won't go out of bounds
    
    # Checks bottom left
    if(new_rank != 0 and new_file != 0 and game_state[new_rank-1][new_file-1] == king):
        old_loc = str(new_rank-1) + str(new_file-1)
    # Checks left
    elif(new_file != 0 and game_state[new_rank][new_file-1] == king):
        old_loc = str(new_rank) + str(new_file-1)
    # Checks top left
    elif(new_rank != 7 and new_file != 0 and game_state[new_rank+1][new_file-1] == king):
        old_loc = str(new_rank + 1) + str(new_file - 1)
    # Checks bottom
    elif(new_rank != 0 and game_state[new_rank-1][new_file] == king):
        old_loc = str(new_rank-1) + str(new_file)
    # Checks top
    elif(new_rank != 7 and game_state[new_rank+1][new_file] == king):
        old_loc = str(new_rank+1) + str(new_file)
    # Checks bottom right
    elif(new_rank != 0 and new_file != 7 and game_state[new_rank-1][new_file+1] == king):
        old_loc = str(new_rank-1) + str(new_file+1)
    # Checks right
    elif(new_file != 7 and game_state[new_rank][new_file+1] == king):
        old_loc = str(new_rank) + str(new_file+1)
    # Checks top right
    else:
        old_loc = str(new_rank+1) + str(new_file+1)
    
    return old_loc
    

# Next we create a helper method for King movement

# isWhite is a boolean value to see which color is moving
# move is the string of the move in Algebraic Notation
# game_state is the board
def move_king(isWhite, move, game_state):
    if(isWhite):
        king = 6
    else:
        king = 12
    
    loc = find_king(isWhite, move, game_state)
    old_rank = int(loc[0])
    old_file = int(loc[1])
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
    
    game_state[old_rank][old_file] *= -1
    game_state[ord(move[1])-49][ord(move[0])-97] = king
    
    return game_state

In [8]:
# Create a helper method to find the Rook in question

def find_rook(isWhite, move, game_state):
    if(isWhite):
        rook = 4
    else:
        rook = 10
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
    
    # Check to see if the two rooks are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        # Check to see if it claims a piece
        if(move[1] == "x"):
            move = move[0] + move[2:]
        
        # This would mean the two rooks can get on the given spot and are both on the same file
        loc = move[1] + move[0]
        
        
    else:
        # Still possibly ambiguous, must check further
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104 or move[1] == "x"):
            # Check to see if it claims a piece
            if(move[1] == "x"):
                move = move[0] + move[2:]
                
            # This is ambiguous, and both rooks are not on the same file
            for r in range(8):
                if(game_state[r][ord(move[0])-97] == rook):
                    loc = move[0]+chr(r+49)
        else:
            # Not ambiguous, we must now find where the rook comes from
            file = ord(move[0])-97
            rank = ord(move[1])-49
            
            loc = "NA"
            
            # Check left of final spot
            if(file != 0):
                f = file
                while f > 0 and loc == "NA":
                    f -= 1
                    if(game_state[rank][f] == rook):
                        loc = chr(f+97) + chr(rank+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[rank][f] != 0):
                        break
            
            # Check right of final spot
            if(file != 7):
                f = file
                while f < 7 and loc == "NA":
                    f += 1
                    if(game_state[rank][f] == rook):
                        loc = chr(f+97)+ chr(rank+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[rank][f] != 0):
                        break
            
            # Check above the final spot
            if(rank != 7):
                r = rank
                while r < 7 and loc == "NA":
                    r += 1
                    if(game_state[r][file] == rook):
                        loc = chr(file+97) + chr(r+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[r][file] != 0):
                        break
                        
            # Check below the final spot
            if(rank != 0):
                r = rank
                while r > 0 and loc == "NA":
                    r -= 1
                    if(game_state[r][file] == rook):
                        loc = chr(file+97) + chr(r+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[r][file] != 0):
                        break
                        
    return loc

def move_rook(isWhite, move, game_state):
    if(isWhite):
        rook = 4
    else:
        rook = 10
        
    # Get original location
    loc = find_rook(isWhite, move, game_state)
    file = ord(loc[0])-97
    rank = ord(loc[1])-49
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    if(move[1] == "x"):
        move = move[0] + move[2:]
    
    # Get new location
    # Check to see if the two rooks are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        new_rank = ord(move[2]) - 49
        new_file = ord(move[1]) - 97
    else:
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104):
            new_rank = ord(move[2]) - 49
            new_file = ord(move[1]) - 97
        else:
            new_rank = ord(move[1]) - 49
            new_file = ord(move[0]) - 97
        
    game_state[rank][file] *= -1
    game_state[new_rank][new_file] = rook
    
    return game_state

In [9]:
# Create a helper method to find the Bishop in question

def find_bishop(isWhite, move, game_state):
    if(isWhite):
        bishop = 3
    else:
        bishop = 9
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    loc = "NA"
        
    # Check to see if the two bishops are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        # Check to see if it claims a piece
        if(move[1] == "x"):
            move = move[0] + move[2:]
        
        # This would mean the two bishops can get on the given spot and are both on the same file
        # Do this on the basis that the bishop must move in a diagonal only
        distance = ord(move[2]) - ord(move[0])
        
        # Now look for which file the bishop is coming from
        if(ord(move[1]) - 97 - distance < 0):
            old_file = chr(ord(move[1]) + distance)
        elif(ord(move[1]) - 97 + distance > 7):
            old_file = chr(ord(move[1])) - distance
        else:
            if(game_state[ord(move[0])-49][ord(move[1])-97-distance] == bishop):
                old_file = chr(ord(move[1]) - distance)
            else:
                old_file = chr(ord(move[1]) + distance)
        loc = old_file + move[0]
    else:
        # Still possibly ambiguous, must check further
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104 or move[1] == "x"):
            # Check to see if it claims a piece
            if(move[1] == "x"):
                move = move[0] + move[2:]
            
            for r in range(7):
                if(game_state[r][ord(move[1])-97] == bishop):
                    rank = chr(r+49)
            loc = move[1] + rank
        # Only one possible bishop
        else:
            # Gets codified indices
            file = ord(move[0]) - 97
            rank = ord(move[1]) - 49
            
            # Check the top left of the current space
            if(file != 0 or rank != 7):
                step = 0
                while file - step > 0 and rank + step < 7 and loc == "NA":
                    step += 1
                    if(game_state[rank+step][file-step] == bishop):
                        loc = chr(file-step+97) + chr(rank+step+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[rank+step][file-step] != 0):
                        break
                        
            # Check the bottom left of the current space
            if(file != 0 or rank != 0):
                step = 0
                while file - step > 0 and rank - step > 0 and loc == "NA":
                    step += 1
                    if(game_state[rank-step][file-step] == bishop):
                        loc = chr(file-step+97) + chr(rank-step+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[rank-step][file-step] != 0):
                        break
                        
            # Check the top right of the current space
            if(file != 7 or rank != 7):
                step = 0
                while file + step < 7 and rank + step < 7 and loc == "NA":
                    step += 1
                    if(game_state[rank+step][file+step] == bishop):
                        loc = chr(file+step+97) + chr(rank+step+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[rank+step][file+step] != 0):
                        break
            
            # Check the bottom right of the current space
            if(file != 7 or rank != 0):
                step = 0
                while file + step < 7 and rank - step > 0 and loc == "NA":
                    step += 1
                    if(game_state[rank-step][file+step] == bishop):
                        loc = chr(file+step+97) + chr(rank-step+49)
                    
                    # Stop if you hit a non-empty space
                    if(game_state[rank-step][file+step] != 0):
                        break
            
    return loc

# Now makes the function that moves the bishop
def move_bishop(isWhite, move, game_state):
    if(isWhite):
        bishop = 3
    else:
        bishop = 9
        
    # Get original location
    loc = find_bishop(isWhite, move, game_state)
    file = ord(loc[0])-97
    rank = ord(loc[1])-49
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    if(move[1] == "x"):
        move = move[0] + move[2:]
        
    # Get new location
    # Check to see if the two rooks are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        new_rank = ord(move[2]) - 49
        new_file = ord(move[1]) - 97
    else:
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104):
            new_rank = ord(move[2]) - 49
            new_file = ord(move[1]) - 97
        else:
            new_rank = ord(move[1]) - 49
            new_file = ord(move[0]) - 97
    
    game_state[rank][file] *= -1
    game_state[new_rank][new_file] = bishop
    
    return game_state

In [10]:
# Create a helper method to find the Knight in question

def find_knight(isWhite, move, game_state):
    if(isWhite):
        knight = 2
    else:
        knight = 8
        
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    loc = "NA"
    
    # Check to see if the two knights are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        # Check to see if it claims a piece
        if(move[1] == "x"):
            move = move[0] + move[2:]
        
        # This would mean the two knights can get on the given spot and are both on the same file
        new_file = ord(move[1])-97
        new_rank = ord(move[2])-49
        
        # Check the possible areas where the knight can come from the specific rank
        # 2 Up, 1 Left
        if(new_file != 0 and new_rank < 6):
            if(game_state[new_rank+2][new_file-1] == knight and new_rank+2 == ord(move[0])-49):
                loc = chr(new_file-1+97) + chr(new_rank+2+49)
                
        # 2 Up, 1 Right
        if(new_file != 7 and new_rank < 6 and loc == "NA"):
            if(game_state[new_rank+2][new_file+1] == knight and new_rank+2 == ord(move[0])-49):
                loc = chr(new_file+1+97) + chr(new_rank+2+49)
                
        # 1 Up, 2 Left
        if(new_file > 1 and new_rank != 7 and loc == "NA"):
            if(game_state[new_rank+1][new_file-2] == knight and new_rank+1 == ord(move[0])-49):
                loc = chr(new_file-2+97) + chr(new_rank+1+49)
                
        # 1 Up, 2 Right
        if(new_file < 6 and new_rank != 7 and loc == "NA"):
            if(game_state[new_rank+1][new_file+2] == knight and new_rank+1 == ord(move[0])-49):
                loc = chr(new_file+2+97) + chr(new_rank+1+49)
            
        # 1 Down, 2 Left
        if(new_file > 1 and new_rank != 0 and loc == "NA"):
            if(game_state[new_rank-1][new_file-2] == knight and new_rank-1 == ord(move[0])-49):
                loc = chr(new_file-2+97) + chr(new_rank-1+49)
                
        # 1 Down, 2 Right
        if(new_file < 6 and new_rank != 0 and loc == "NA"):
            if(game_state[new_rank-1][new_file+2] == knight and new_rank-1 == ord(move[0])-49):
                loc = chr(new_file+2+97) + chr(new_rank-1+49)
                
        # 2 Down, 1 Left
        if(new_file != 0 and new_rank > 1 and loc == "NA"):
            if(game_state[new_rank-2][new_file-1] == knight and new_rank-2 == ord(move[0])-49):
                loc = chr(new_file-1+97) + chr(new_rank-2+49)
                
        # 2 Down, 1 Right
        if(new_file != 7 and new_rank > 1 and loc == "NA"):
            if(game_state[new_rank-2][new_file+1] == knight and new_rank-2 == ord(move[0])-49):
                loc = chr(new_file+1+97) + chr(new_rank-2+49)
    else:
        # Still ambiguous
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104 or move[1] == "x"):
            # Check to see if it claims a piece
            if(move[1] == "x"):
                move = move[0] + move[2:]
            
             # This would mean the two knights can get on the given spot and are both on the same file
            new_file = ord(move[1])-97
            new_rank = ord(move[2])-49
            
            # Check the possible areas where the knight can come from the specific file
            
            # 2 Up, 1 Left
            if(new_file != 0 and new_rank < 6):
                if(game_state[new_rank+2][new_file-1] == knight and new_file-1 == ord(move[0])-97):
                    loc = chr(new_file-1+97) + chr(new_rank+2+49)


            # 2 Up, 1 Right
            if(new_file != 7 and new_rank < 6 and loc == "NA"):
                if(game_state[new_rank+2][new_file+1] == knight and new_file+1 == ord(move[0])-97):
                    loc = chr(new_file+1+97) + chr(new_rank+2+49)

            # 1 Up, 2 Left
            if(new_file > 1 and new_rank != 7 and loc == "NA"):
                if(game_state[new_rank+1][new_file-2] == knight and new_file-2 == ord(move[0])-97):
                    loc = chr(new_file-2+97) + chr(new_rank+1+49)

            # 1 Up, 2 Right
            if(new_file < 6 and new_rank != 7 and loc == "NA"):
                if(game_state[new_rank+1][new_file+2] == knight and new_file+2 == ord(move[0])-97):
                    loc = chr(new_file+2+97) + chr(new_rank+1+49)

            # 1 Down, 2 Left
            if(new_file > 1 and new_rank != 0 and loc == "NA"):
                if(game_state[new_rank-1][new_file-2] == knight and new_file-2 == ord(move[0])-97):
                    loc = chr(new_file-2+97) + chr(new_rank-1+49)

            # 1 Down, 2 Right
            if(new_file < 6 and new_rank != 0 and loc == "NA"):
                if(game_state[new_rank-1][new_file+2] == knight and new_file+2 == ord(move[0])-97):
                    loc = chr(new_file+2+97) + chr(new_rank-1+49)

            # 2 Down, 1 Left
            if(new_file != 0 and new_rank > 1 and loc == "NA"):
                if(game_state[new_rank-2][new_file-1] == knight and new_file-1 == ord(move[0])-97):
                    loc = chr(new_file-1+97) + chr(new_rank-2+49)

            # 2 Down, 1 Right
            if(new_file != 7 and new_rank > 1 and loc == "NA"):
                if(game_state[new_rank-2][new_file+1] == knight and new_file+1 == ord(move[0])-97):
                    loc = chr(new_file+1+97) + chr(new_rank-2+49)
        else:
            # Only one possible knight
            # This would mean the two knights can get on the given spot and are both on the same file
            new_file = ord(move[0])-97
            new_rank = ord(move[1])-49
            
            # 2 Up, 1 Left
            if(new_file != 0 and new_rank < 6):
                if(game_state[new_rank+2][new_file-1] == knight):
                    loc = chr(new_file-1+97) + chr(new_rank+2+49)

            # 2 Up, 1 Right
            if(new_file != 7 and new_rank < 6 and loc == "NA"):
                if(game_state[new_rank+2][new_file+1] == knight):
                    loc = chr(new_file+1+97) + chr(new_rank+2+49)

            # 1 Up, 2 Left
            if(new_file > 1 and new_rank != 7 and loc == "NA"):
                if(game_state[new_rank+1][new_file-2] == knight):
                    loc = chr(new_file-2+97) + chr(new_rank+1+49)

            # 1 Up, 2 Right
            if(new_file < 6 and new_rank != 7 and loc == "NA"):
                if(game_state[new_rank+1][new_file+2] == knight):
                    loc = chr(new_file+2+97) + chr(new_rank+1+49)

            # 1 Down, 2 Left
            if(new_file > 1 and new_rank != 0 and loc == "NA"):
                if(game_state[new_rank-1][new_file-2] == knight):
                    loc = chr(new_file-2+97) + chr(new_rank-1+49)

            # 1 Down, 2 Right
            if(new_file < 6 and new_rank != 0 and loc == "NA"):
                if(game_state[new_rank-1][new_file+2] == knight):
                    loc = chr(new_file+2+97) + chr(new_rank-1+49)

            # 2 Down, 1 Left
            if(new_file != 0 and new_rank > 1 and loc == "NA"):
                if(game_state[new_rank-2][new_file-1] == knight):
                    loc = chr(new_file-1+97) + chr(new_rank-2+49)

            # 2 Down, 1 Right
            if(new_file != 7 and new_rank > 1 and loc == "NA"):
                if(game_state[new_rank-2][new_file+1] == knight):
                    loc = chr(new_file+1+97) + chr(new_rank-2+49)
        
    return loc
                
# Creates the function to actually move the knights
def move_knight(isWhite, move, game_state):
    if(isWhite):
        knight = 2
    else:
        knight = 8
        
    # Get new location        
    loc = find_knight(isWhite, move, game_state)
    file = ord(loc[0])-97
    rank = ord(loc[1])-49
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    if(move[1] == "x"):
        move = move[0] + move[2:]

    # Check to see if the two rooks are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        new_rank = ord(move[2]) - 49
        new_file = ord(move[1]) - 97
    else:
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104):
            new_rank = ord(move[2]) - 49
            new_file = ord(move[1]) - 97
        else:
            new_rank = ord(move[1]) - 49
            new_file = ord(move[0]) - 97
    
    game_state[rank][file] *= -1
    game_state[new_rank][new_file] = knight
    
    return game_state

In [11]:
# Create a helper method to find the Queen in question

def find_queen(isWhite, move, game_state):
    if(isWhite):
        queen = 5
    else:
        queen = 11
        
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    loc = "NA"
    
    # Check to see if the two queens are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        # Check to see if it claims a piece
        if(move[1] == "x"):
            move = move[0] + move[2:]
        
        # This would mean the two queens can get on the given spot and are both on the same file
        # Find codified new_rank and new_file
        correct_rank = ord(move[0]) - 49
        file = ord(move[1]) - 97
        rank = ord(move[2]) - 49
        
        # Check the top left of the current space
        if(file != 0 or rank != 7):
            step = 0
            while file - step > 0 and rank + step < 7 and loc == "NA":
                step += 1
                if(game_state[rank+step][file-step] == queen and rank+step == correct_rank):
                    loc = chr(file-step+97) + chr(rank+step+49)

                # Stop if you hit a non-empty space
                if(game_state[rank+step][file-step] != 0):
                    break

        # Check the bottom left of the current space
        if(file != 0 or rank != 0):
            step = 0
            while file - step > 0 and rank - step > 0 and loc == "NA":
                step += 1
                if(game_state[rank-step][file-step] == queen and rank-step == correct_rank):
                    loc = chr(file-step+97) + chr(rank-step+49)

                # Stop if you hit a non-empty space
                if(game_state[rank-step][file-step] != 0):
                    break

        # Check the top right of the current space
        if(file != 7 or rank != 7):
            step = 0
            while file + step < 7 and rank + step < 7 and loc == "NA":
                step += 1
                if(game_state[rank+step][file+step] == queen and rank+step == correct_rank):
                    loc = chr(file+step+97) + chr(rank+step+49)

                # Stop if you hit a non-empty space
                if(game_state[rank+step][file+step] != 0):
                    break

        # Check the bottom right of the current space
        if(file != 7 or rank != 0):
            step = 0
            while file + step < 7 and rank - step > 0 and loc == "NA":
                step += 1
                if(game_state[rank-step][file+step] == queen and rank-step == correct_rank):
                    loc = chr(file+step+97) + chr(rank-step+49)

                # Stop if you hit a non-empty space
                if(game_state[rank-step][file+step] != 0):
                    break
        
        # Check left of final spot
        if(file != 0):
            f = file
            while f > 0 and loc == "NA":
                f -= 1
                if(game_state[rank][f] == queen and rank == correct_rank):
                    loc = chr(f+97) + chr(rank+49)

                # Stop if you hit a non-empty space
                if(game_state[rank][f] != 0):
                    break

        # Check right of final spot
        if(file != 7):
            f = file
            while f < 7 and loc == "NA":
                f += 1
                if(game_state[rank][f] == queen and rank == correct_rank):
                    loc = chr(f+97)+ chr(rank+49)

                # Stop if you hit a non-empty space
                if(game_state[rank][f] != 0):
                    break

        # Check above the final spot
        if(rank != 7):
            r = rank
            while r < 7 and loc == "NA":
                r += 1
                if(game_state[r][file] == queen and r == correct_rank):
                    loc = chr(file+97) + chr(r+49)

                # Stop if you hit a non-empty space
                if(game_state[r][file] != 0):
                    break

        # Check below the final spot
        if(rank != 0):
            r = rank
            while r > 0 and loc == "NA":
                r -= 1
                if(game_state[r][file] == queen and r == correct_rank):
                    loc = chr(file+97) + chr(r+49)

                # Stop if you hit a non-empty space
                if(game_state[r][file] != 0):
                    break
    else:
        # Still ambiguous, but uses file to disambiguate
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104 or move[1] == "x"):
            # Check to see if it claims a piece
            if(move[1] == "x"):
                move = move[0] + move[2:]
            
            correct_file = ord(move[0]) - 97
            file = ord(move[1]) - 97
            rank = ord(move[2]) - 49

            # Check the top left of the current space
            if(file != 0 or rank != 7):
                step = 0
                while file - step > 0 and rank + step < 7 and loc == "NA":
                    step += 1
                    if(game_state[rank+step][file-step] == queen and file-step == correct_file):
                        loc = chr(file-step+97) + chr(rank+step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank+step][file-step] != 0):
                        break

            # Check the bottom left of the current space
            if(file != 0 or rank != 0):
                step = 0
                while file - step > 0 and rank - step > 0 and loc == "NA":
                    step += 1
                    if(game_state[rank-step][file-step] == queen and file-step == correct_file):
                        loc = chr(file-step+97) + chr(rank-step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank-step][file-step] != 0):
                        break

            # Check the top right of the current space
            if(file != 7 or rank != 7):
                step = 0
                while file + step < 7 and rank + step < 7 and loc == "NA":
                    step += 1
                    if(game_state[rank+step][file+step] == queen and file+step == correct_file):
                        loc = chr(file+step+97) + chr(rank+step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank+step][file+step] != 0):
                        break

            # Check the bottom right of the current space
            if(file != 7 or rank != 0):
                step = 0
                while file + step < 7 and rank - step > 0 and loc == "NA":
                    step += 1
                    if(game_state[rank-step][file+step] == queen and file+step == correct_file):
                        loc = chr(file+step+97) + chr(rank-step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank-step][file+step] != 0):
                        break

            # Check left of final spot
            if(file != 0):
                f = file
                while f > 0 and loc == "NA":
                    f -= 1
                    if(game_state[rank][f] == queen and f == correct_file):
                        loc = chr(f+97) + chr(rank+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank][f] != 0):
                        break

            # Check right of final spot
            if(file != 7):
                f = file
                while f < 7 and loc == "NA":
                    f += 1
                    if(game_state[rank][f] == queen and f == correct_file):
                        loc = chr(f+97)+ chr(rank+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank][f] != 0):
                        break

            # Check above the final spot
            if(rank != 7):
                r = rank
                while r < 7 and loc == "NA":
                    r += 1
                    if(game_state[r][file] == queen and file == correct_file):
                        loc = chr(file+97) + chr(r+49)

                    # Stop if you hit a non-empty space
                    if(game_state[r][file] != 0):
                        break

            # Check below the final spot
            if(rank != 0):
                r = rank
                while r > 0 and loc == "NA":
                    r -= 1
                    if(game_state[r][file] == queen and file == correct_file):
                        loc = chr(file+97) + chr(r+49)

                    # Stop if you hit a non-empty space
                    if(game_state[r][file] != 0):
                        break
        else:
            # Completely unambiguous, just do the previous tests but no need to check file or rank
            file = ord(move[0]) - 97
            rank = ord(move[1]) - 49

            # Check the top left of the current space
            if(file != 0 or rank != 7):
                step = 0
                while file - step > 0 and rank + step < 7 and loc == "NA":
                    step += 1
                    if(game_state[rank+step][file-step] == queen):
                        loc = chr(file-step+97) + chr(rank+step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank+step][file-step] != 0):
                        break

            # Check the bottom left of the current space
            if(file != 0 or rank != 0):
                step = 0
                while file - step > 0 and rank - step > 0 and loc == "NA":
                    step += 1
                    if(game_state[rank-step][file-step] == queen):
                        loc = chr(file-step+97) + chr(rank-step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank-step][file-step] != 0):
                        break

            # Check the top right of the current space
            if(file != 7 or rank != 7):
                step = 0
                while file + step < 7 and rank + step < 7 and loc == "NA":
                    step += 1
                    if(game_state[rank+step][file+step] == queen):
                        loc = chr(file+step+97) + chr(rank+step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank+step][file+step] != 0):
                        break

            # Check the bottom right of the current space
            if(file != 7 or rank != 0):
                step = 0
                while file + step < 7 and rank - step > 0 and loc == "NA":
                    step += 1
                    if(game_state[rank-step][file+step] == queen):
                        loc = chr(file+step+97) + chr(rank-step+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank-step][file+step] != 0):
                        break

            # Check left of final spot
            if(file != 0):
                f = file
                while f > 0 and loc == "NA":
                    f -= 1
                    if(game_state[rank][f] == queen):
                        loc = chr(f+97) + chr(rank+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank][f] != 0):
                        break

            # Check right of final spot
            if(file != 7):
                f = file
                while f < 7 and loc == "NA":
                    f += 1
                    if(game_state[rank][f] == queen):
                        loc = chr(f+97)+ chr(rank+49)

                    # Stop if you hit a non-empty space
                    if(game_state[rank][f] != 0):
                        break

            # Check above the final spot
            if(rank != 7):
                r = rank
                while r < 7 and loc == "NA":
                    r += 1
                    if(game_state[r][file] == queen):
                        loc = chr(file+97) + chr(r+49)

                    # Stop if you hit a non-empty space
                    if(game_state[r][file] != 0):
                        break

            # Check below the final spot
            if(rank != 0):
                r = rank
                while r > 0 and loc == "NA":
                    r -= 1
                    if(game_state[r][file] == queen):
                        loc = chr(file+97) + chr(r+49)

                    # Stop if you hit a non-empty space
                    if(game_state[r][file] != 0):
                        break
                        
    return loc

# Creates a private helper method that moves the queen
def move_queen(isWhite, move, game_state):
    if(isWhite):
        queen = 5
    else:
        queen = 11
        
    # Gets new location
    loc = find_queen(isWhite, move, game_state)
    file = ord(loc[0])-97
    rank = ord(loc[1])-49
    
    move = move[1:]
    
    if(move[0] == "x"):
        move = move[1:]
        
    if(move[1] == "x"):
        move = move[0] + move[2:]
    
    # Check to see if the two queens are on the same file
    if(ord(move[0]) >= 49 and ord(move[0]) <= 56):
        new_rank = ord(move[2]) - 49
        new_file = ord(move[1]) - 97
    else:
        if(ord(move[1]) >= 97 and ord(move[1]) <= 104):
            new_rank = ord(move[2]) - 49
            new_file = ord(move[1]) - 97
        else:
            new_rank = ord(move[1]) - 49
            new_file = ord(move[0]) - 97
    
    game_state[rank][file] *= -1
    game_state[new_rank][new_file] = queen
    
    return game_state

In [12]:
# Creates function for castling
def move_castle(isWhite, move, gamestate):
    
    # Checks if it is a King's side castling
    if move == "O-O":
        
        if (isWhite):
            king = 6
            rook = 4
            # Moves all the pieces as necessary on white's turn
            gamestate[0][4] *= -1
            gamestate[0][7] *= -1
            gamestate[0][6] = king
            gamestate[0][5] = rook
            
        else:
            king = 12
            rook = 10
            # Moves all the pieces as necessary on black's turn
            gamestate[7][4] *= -1
            gamestate[7][7] *= -1
            gamestate[7][6] = king
            gamestate[7][5] = rook
            
    # Else it is a Queen's side castling
    else:
        if (isWhite):
            king = 6
            rook = 4
            # Moves all the pieces as necessary on white's turn
            gamestate[0][4] *= -1
            gamestate[0][0] *= -1
            gamestate[0][2] = king
            gamestate[0][3] = rook
            
        else:
            king = 12
            rook = 10
            # Moves all the pieces as necessary on black's turn
            gamestate[7][4] *= -1
            gamestate[7][0] *= -1
            gamestate[7][2] = king
            gamestate[7][3] = rook
    
    return gamestate
        

In [13]:
# Private method to remove negative values from codified matrix

def removeNegatives(game_state):
    for i in range(8):
        for j in range(8):
            if(game_state[i][j] < 0):
                game_state[i][j] = 0
    return game_state

In [14]:
# Private method to serialize an Algebraic Notated chess game
def serialize(alg_not):
    # Gets the annotated game
    game = alg_not
    
    # Splits the string into a list
    game = game.split()
    
    # Removes the ending of the list which is an unnecessary data point for us
    game = game[:-1]
    
    ret_game = []
    # Removes the turn numbers from the annotated version
    for i in range(len(game)):
        # A turn indicator appears every third value
        if(i%3 != 0):
            ret_game.append(game[i])
    
    return ret_game

In [15]:
# A method to play through an annotated text
def playGame(annotation, showState = False):
    gs = setUpGame()
    isWhite = True
    
    # Plays through every move done
    for turn in range(len(annotation)):
        
        if(showState):
            print(annotation[turn])
            print(isWhite)
        
        # Get rid of any negative values
        gs = removeNegatives(gs)
        
        # Finds which piece is being moved and plays the turn
        start_char = annotation[turn][0]
        if(start_char == "Q"):
            gs = move_queen(isWhite, annotation[turn], gs)
        elif(start_char == "N"):
            gs = move_knight(isWhite, annotation[turn], gs)
        elif(start_char == "B"):
            gs = move_bishop(isWhite, annotation[turn], gs)
        elif(start_char == "R"):
            gs = move_rook(isWhite, annotation[turn], gs)
        elif(start_char == "K"):
            gs = move_king(isWhite, annotation[turn], gs)
        elif(start_char == "O"):
            gs = move_castle(isWhite, annotation[turn], gs)
        else:
            gs = move_pawn(isWhite, annotation[turn], gs)
            
        # Swaps to other colors turn
        isWhite = not isWhite
        
        if(showState):
            print(gs)
            print()

In [16]:
# Test through a whole game

play1 = chess_clean["AN"]
play1 = play1.iloc[0]

play = serialize(play1)

print(play1)
playGame(play, True)

1. e4 e5 2. Bc4 Nc6 3. Nf3 Bc5 4. c3 Nf6 5. d4 exd4 6. cxd4 Nxe4 7. dxc5 Qe7 8. O-O Nxc5 9. Bg5 f6 10. Re1 Ne5 11. Nxe5 Rf8 12. Nxd7 Nxd7 13. Rxe7+ Kxe7 14. Qe1+ Kd8 15. Bf4 Re8 16. Qc3 Nc5 17. Bf7 Ne4 18. Qxc7# 1-0
e4
True
[[ 4.  2.  3.  5.  6.  3.  2.  4.]
 [ 1.  1.  1.  1. -1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 7.  7.  7.  7.  7.  7.  7.  7.]
 [10.  8.  9. 11. 12.  9.  8. 10.]]

e5
False
[[ 4.  2.  3.  5.  6.  3.  2.  4.]
 [ 1.  1.  1.  1.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  7.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 7.  7.  7.  7. -7.  7.  7.  7.]
 [10.  8.  9. 11. 12.  9.  8. 10.]]

Bc4
True
[[ 4.  2.  3.  5.  6. -3.  2.  4.]
 [ 1.  1.  1.  1.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  3.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  7.  0.  0.  0.]
 [ 0.  0.  0.  0.  